# LLM Definition retrieval and generation agent with LangGraph

In [ ]:
%pip install -qU \
    datasets==2.19.1 \
    langchain-pinecone==0.1.1 \
    langchain-openai==0.1.9 \
    langchain==0.2.5 \
    langchain-core==0.2.9 \
    langgraph==0.1.1 \
    semantic-router==0.0.48 \
    serpapi==0.1.5 \
    google-search-results==2.4.2 \
    pygraphviz==1.12  # for visualizing

## Research Agent Overview

The research agent will consist of a function calling AI agent that has access to several tools that it can use to find information on a particular topic. 
It will be able to use several tools over multiple steps, meaning it can find information on one topic, broaden the scope of knowledge on this topic and _even_ investigate parallel topics where relevant.

The tools we will be using are:

* **RAG search**: We will create a vector knowledge base containing the embeddings of documents from the datasets. This tool provides our agent with access to this knowledge. [*pinecone*, *faiss*]
* **RAG search with filter**: The agent may need more information from a specific document, this tool allows our agent to do just that. [*opensearch*]
* **Reference resolution**: If a definition contains a reference, this tool enable the agent to resolve it.
* **Web search**: This tool provides our agent with access to online thesaurus and standardized vocabularies (e.g. https://op.europa.eu/it/web/eu-vocabularies)
* **Final answer**: We create a custom final answer tool that forces our agent to output information in a specific format.

## Graph State

We will define a custom graph state to support our agent-oriented decision making.

In [3]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

There are four parts to our agent state, those are:

* `input`: this is the user's most recent query, usually this would be a question that we want to answer with our research agent.
* `chat_history`: we are building a conversational agent that can support multiple interactions, to allow previous interactions to provide additional context throughout our agent logic we include the chat history in the agent state.
* `intermediate_steps`: provides a record of all steps the research agent will take between the user asking a question via `input` and the agent providing a final answer. This can include things like "search arxiv", "perform general purpose web search", etc. These intermediate steps are crucial to allowing the agent to follow a path of coherent actions and ultimately producing an informed final answer

## Tools

### RAG search

We provide two RAG-focused tools for our agent. The `rag_search` allows the agent to perform a simple RAG search for some information across _all_ indexed documents, retrieving the most semantically similar documents based on a similarity score.

The `rag_opensearch` searches instead _within_ the documents in the dataset for a specific string contained within the document. This allows the agent to find specific information within a document.

#### Vector Knowledge Base Setup

We encode the documents into a vector space using a pre-trained model. This base will be used to find the most similar documents to a given query.

<div class="alert alert-block alert-warning">
Which documents do we encode? The whole dataset? Only the documents with definitions?
</div>
<div class="alert alert-block alert-warning">
What do we encode? The whole documents? The keywords? The definitions?
</div>
<div class="alert alert-block alert-warning">
How do we encode the documents? What model do we use?
</div>

#### RAG Search

### Reference Resolution

## Controller

The **Controller** is our graph's decision maker. It decides which path we should take down our graph. It functions similarly to an agent but is much simpler and reliable.

The Controller consists of an LLM provided with a set of potential function calls (i.e. our tools) that it can decide to use — we force it to use _at least_ one of those tool using the `tool_choice="any"` setting (see below). Our Controller only makes the decision to use a tool, it doesn't execute the tool code itself (we do that seperately in our graph)

### Controller Prompt

Our prompt for the Oracle will emphasize it's decision making ability within the `system_prompt`, leave a placeholder for us to later insert `chat_history`, and provide a place for us to insert the user `input`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt = """You are the oracle, the great AI decision maker.
Given the user's query you must decide what to do with it based on the
list of tools provided to you.

If you see that a tool has been used (in the scratchpad) with a particular
query, do NOT use that same tool with the same query again. Also, do NOT use
any tool more than twice (ie, if the tool appears in the scratchpad twice, do
not use it again).

You should aim to collect information from a diverse range of sources before
providing the answer to the user. Once you have collected plenty of information
to answer the user's question (stored in the scratchpad) use the final_answer
tool."""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("assistant", "scratchpad: {scratchpad}"),
])

Next, we must initialize our `llm` (for this we use `gpt-4o`) and then create the _runnable_ pipeline of our Oracle.

The runnable connects our inputs (the user `input` and `chat_history`) to our `prompt`, and our `prompt` to our `llm`. It is also where we _bind_ our tools to the LLM and enforce function calling via `tool_choice="any"`.

In [ ]:
from langchain_core.messages import ToolCall, ToolMessage
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    temperature=0
)

tools=[
    rag_search_filter,
    rag_search,
    fetch_arxiv,
    web_search,
    final_answer
]

# define a function to transform intermediate_steps from list
# of AgentAction to scratchpad string
def create_scratchpad(intermediate_steps: list[AgentAction]):
    research_steps = []
    for i, action in enumerate(intermediate_steps):
        if action.log != "TBD":
            # this was the ToolExecution
            research_steps.append(
                f"Tool: {action.tool}, input: {action.tool_input}\n"
                f"Output: {action.log}"
            )
    return "\n---\n".join(research_steps)

oracle = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "scratchpad": lambda x: create_scratchpad(
            intermediate_steps=x["intermediate_steps"]
        ),
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

## Evaluation

### Quantitative

### Qualitative

---